In [62]:
import pandas as pd 
import numpy as np
from scipy.special import comb
import math
from neal import SimulatedAnnealingSampler
from pyqubo import Array, Constraint, Placeholder, solve_qubo
import itertools
import random
import matplotlib.pyplot as plt

### 解析用データ作成

（3年間骨肉腫に罹患しない）事象と各特徴量とのP値を計算する。
- Osteogenic sarcoma data:骨肉腫
- LI:リンパ球浸出
- AOP:類骨病理学

In [63]:
ost16 = pd.read_csv("../input/ost16.csv", sep=',', index_col=0)
ost16.shape

(16, 4)

In [64]:
#indexを振り直す
ost16 = ost16.reset_index(drop=True)

In [65]:
ost16

,Y,LI,SEX,AOP
0,0,1,0,1
1,1,1,0,1
2,1,1,0,0
3,1,1,0,1
4,0,1,1,0
5,1,0,1,0
6,1,0,1,0
7,1,1,1,0
8,1,1,1,1
9,1,1,1,1


In [66]:
t_list = []
t_list.append(sum(ost16['Y']))
print(t_list)
t_list.append(np.dot(ost16['Y'], ost16['LI']))
t_list.append(np.dot(ost16['Y'], ost16['SEX']))
t_list.append(np.dot(ost16['Y'], ost16['AOP']))

[14]


In [67]:
t_list#Y, LI, SEX, AOP

[14, 9, 9, 8]

In [68]:
columns = list(ost16.columns)
num_samples = ost16.shape[0]

In [69]:
#!pip3 install dwave-system

In [70]:
from dwave.system.composites import AutoEmbeddingComposite
from dwave.system.samplers import DWaveSampler
from pyqubo import Array, Constraint, Placeholder, solve_qubo

In [71]:
DWAVE_TOKEN="TOKY-1319d5c52b9aa35f34b40feba0cea58a4f5d3c09" #"replace this string with actual token"

In [72]:
t1 = 9

In [73]:
y = Array.create('y', shape=ost16.shape[0], vartype='BINARY')

#QUBO式で定式化
H = (sum(y) - t_list[0])**2
H += (np.dot(ost16[columns[1]], y) - t1)**2
for j in range(2, len(columns)):
    H_plus = (np.dot(ost16[columns[j]], y) - t_list[j])**2
    H += H_plus

qb = H.compile().to_qubo()

In [164]:
def t1_9_find_valid_y(res):
    valid_y_list = []
    valid_y_num_t1 = 0
    for y_info in list(res.record):
        y_list = list(y_info[0])
        print(y_list)
        if sum(y_list) == t_list[0]:#int同士の比較
            y = pd.Series(y_info[0])
            print('y is ok now')
            if np.dot(ost16[columns[1]], y) == t1:#int同士の比較
                print('LI is ok too')
                for j in range(2, len(columns)):
                    if np.dot(ost16[columns[j]], y) != t_list[j]:#int同士の比較
                        print('drop out')
                        break
                else:
                    if all(y_list != p for p in valid_y_list):
                        valid_y_num_t1 += 1
                        valid_y_list.append(y_list)
                        print('perfect')
    return valid_y_list, valid_y_num_t1

In [ ]:
dw_sampler = DWaveSampler(
    endpoint="https://cloud.dwavesys.com/sapi",
    solver = 'DW_2000Q_6',
    token = DWAVE_TOKEN
)

qa_sampler = AutoEmbeddingComposite(dw_sampler)

## num_reads=10

### anealing_time = 20

In [153]:
res_reads10 = qa_sampler.sample_qubo(qb[0], num_reads=10)
res_reads10

SampleSet(rec.array([([1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1], -420., 2, 0.0625),
           ([1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1], -420., 1, 0.0625),
           ([1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], -419., 1, 0.0625),
           ([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1], -421., 1, 0.0625),
           ([1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1], -421., 1, 0.0625),
           ([1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1], -416., 1, 0.125 ),
           ([1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1], -416., 1, 0.0625),
           ([1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1], -420., 1, 0.125 ),
           ([1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1], -415., 1, 0.0625)],
          dtype=[('sample', 'i1', (16,)), ('energy', '<f8'), ('num_occurrences', '<i8'), ('chain_break_fraction', '<f8')]), ['y[0]', 'y[10]', 'y[11]', 'y[12]', 'y[13]', 'y[14]', 'y[15]', 'y[1]', 'y[2]', 'y[3]', 'y[4]', 'y[5]', 'y[6]', 'y[7]', 'y[8]', 'y[9]'], 

In [154]:
valid_y_list = []
valid_y_num_t1 = 0
for y_info in list(res_reads10.record):
    y_list = list(y_info[0])
    print(y_list)
    if sum(y_list) == t_list[0]:#int同士の比較
        y = pd.Series(y_info[0])
        print('y is ok now')
        if np.dot(ost16[columns[1]], y) == t1:#int同士の比較
            print('LI is ok too')
            for j in range(2, len(columns)):
                if np.dot(ost16[columns[j]], y) != t_list[j]:#int同士の比較
                    print('drop out')
                    break
            else:
                if all(y_list != p for p in valid_y_list):
                    valid_y_num_t1 += 1
                    valid_y_list.append(y_list)
                    print('perfect')

[1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]
y is ok now
LI is ok too
perfect
[1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1]
y is ok now
LI is ok too
drop out
[1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
y is ok now
[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]
y is ok now
LI is ok too
drop out
[1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]
y is ok now
LI is ok too
perfect
[1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1]
[1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1]
[1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1]


In [155]:
valid_y_num_t1

2

In [156]:
valid_y_list

[[1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1],
 [1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]]

In [157]:
print(np.dot(pd.Series(valid_y_list[0]), ost16['LI']))
print(np.dot(pd.Series(valid_y_list[0]), ost16['SEX']))
print(np.dot(pd.Series(valid_y_list[0]), ost16['AOP']))

9
9
8


In [158]:
print(np.dot(pd.Series(valid_y_list[1]), ost16['LI']))
print(np.dot(pd.Series(valid_y_list[1]), ost16['SEX']))
print(np.dot(pd.Series(valid_y_list[1]), ost16['AOP']))

9
9
8


In [159]:
res_reads10.info

{'timing': {'qpu_sampling_time': 2389,
  'qpu_anneal_time_per_sample': 20,
  'qpu_readout_time_per_sample': 198,
  'qpu_access_time': 13266,
  'qpu_access_overhead_time': 3327,
  'qpu_programming_time': 10877,
  'qpu_delay_time_per_sample': 21,
  'total_post_processing_time': 93,
  'post_processing_overhead_time': 93,
  'total_real_time': 13266,
  'run_time_chip': 2389,
  'anneal_time_per_run': 20,
  'readout_time_per_run': 198},
 'problem_id': 'd60cfaf2-8545-44ff-9ea6-a936da83ba8a'}

### annealing_time=50

In [160]:
res_time50_reads10 = qa_sampler.sample_qubo(qb[0], annealing_time=50, num_reads=10)
res_time50_reads10.info

{'timing': {'qpu_sampling_time': 2689,
  'qpu_anneal_time_per_sample': 50,
  'qpu_readout_time_per_sample': 198,
  'qpu_access_time': 13602,
  'qpu_access_overhead_time': 1490,
  'qpu_programming_time': 10913,
  'qpu_delay_time_per_sample': 21,
  'total_post_processing_time': 99,
  'post_processing_overhead_time': 99,
  'total_real_time': 13602,
  'run_time_chip': 2689,
  'anneal_time_per_run': 50,
  'readout_time_per_run': 198},
 'problem_id': 'cd9cf356-8528-4630-a34a-99e21189315f'}

In [161]:
res_time50_reads10.record

rec.array([([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1], -422., 1, 0.    ),
           ([1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1], -422., 1, 0.    ),
           ([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1], -415., 1, 0.0625),
           ([1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1], -421., 1, 0.    ),
           ([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1], -416., 1, 0.0625),
           ([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1], -420., 1, 0.    ),
           ([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1], -415., 1, 0.125 ),
           ([1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1], -419., 1, 0.    ),
           ([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1], -415., 1, 0.0625),
           ([1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1], -419., 1, 0.0625)],
          dtype=[('sample', 'i1', (16,)), ('energy', '<f8'), ('num_occurrences', '<i8'), ('chain_break_fraction', '<f8')])

In [162]:
valid_y_list = []
valid_y_num_t1 = 0
for y_info in list(res_time50_reads10.record):
    y_list = list(y_info[0])
    print(y_list)
    if sum(y_list) == t_list[0]:#int同士の比較
        y = pd.Series(y_info[0])
        print('y is ok now')
        if np.dot(ost16[columns[1]], y) == t1:#int同士の比較
            print('LI is ok too')
            for j in range(2, len(columns)):
                if np.dot(ost16[columns[j]], y) != t_list[j]:#int同士の比較
                    print('drop out')
                    break
            else:
                if all(y_list != p for p in valid_y_list):
                    valid_y_num_t1 += 1
                    valid_y_list.append(y_list)
                    print('perfect')

[1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1]
y is ok now
[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1]
y is ok now
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1]
[1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1]
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1]
[1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1]
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1]
[1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1]
[1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1]


In [163]:
valid_y_num_t1

0

## num_reads=100

### annealing_time=20

In [208]:
%%timeit

res_reads100 = qa_sampler.sample_qubo(qb[0], num_reads=100)
valid_y_list, valid_y_num_t1 = t1_9_find_valid_y(res_reads100)

[1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]
y is ok now
[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1]
y is ok now
[1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1]
[1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1]
y is ok now
[1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1]
y is ok now
[1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1]
[1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1]
[1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1]
y is ok now
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1]
[1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1]
[0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1]
[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1]
[1, 1, 1,

In [165]:
res_reads100 = qa_sampler.sample_qubo(qb[0], num_reads=100)
res_reads100.info

{'timing': {'qpu_sampling_time': 23894,
  'qpu_anneal_time_per_sample': 20,
  'qpu_readout_time_per_sample': 198,
  'qpu_access_time': 34772,
  'qpu_access_overhead_time': 2852,
  'qpu_programming_time': 10878,
  'qpu_delay_time_per_sample': 21,
  'total_post_processing_time': 220,
  'post_processing_overhead_time': 220,
  'total_real_time': 34772,
  'run_time_chip': 23894,
  'anneal_time_per_run': 20,
  'readout_time_per_run': 198},
 'problem_id': 'f7f56eeb-73ca-4a47-a8ff-0f6b208948e4'}

In [166]:
res_reads100.record

rec.array([([1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1], -419., 3, 0.0625),
           ([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1], -422., 1, 0.    ),
           ([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1], -422., 1, 0.    ),
           ([1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1], -422., 6, 0.    ),
           ([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1], -422., 2, 0.    ),
           ([1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1], -422., 1, 0.    ),
           ([1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1], -413., 1, 0.0625),
           ([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1], -421., 1, 0.    ),
           ([1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1], -421., 1, 0.    ),
           ([1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1], -421., 1, 0.    ),
           ([1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], -421., 1, 0.    ),
           ([1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1], -421., 2, 0.    ),
           ([1, 1, 1, 1, 1, 

In [167]:
valid_y_list, valid_y_num_t1 = t1_9_find_valid_y(res_reads100)

[1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1]
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1]
y is ok now
LI is ok too
perfect
[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1]
y is ok now
LI is ok too
drop out
[1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]
y is ok now
LI is ok too
drop out
[1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1]
y is ok now
LI is ok too
drop out
[1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1]
y is ok now
LI is ok too
drop out
[1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1]
[1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1]
[1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
y is ok now
[1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1]
[1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1]
y is ok now
LI is ok too
perfect
[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
y is ok now
[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,

In [168]:
valid_y_num_t1

2

In [169]:
valid_y_list

[[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
 [1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1]]

### annealing_time=50

In [137]:
res_time50_reads100 = qa_sampler.sample_qubo(qb[0], annealing_time=50, num_reads=100)
res_time50_reads100.info

{'timing': {'qpu_sampling_time': 26894,
  'qpu_anneal_time_per_sample': 50,
  'qpu_readout_time_per_sample': 198,
  'qpu_access_time': 37803,
  'qpu_access_overhead_time': 2340,
  'qpu_programming_time': 10909,
  'qpu_delay_time_per_sample': 21,
  'total_post_processing_time': 644,
  'post_processing_overhead_time': 644,
  'total_real_time': 37803,
  'run_time_chip': 26894,
  'anneal_time_per_run': 50,
  'readout_time_per_run': 198},
 'problem_id': '10f45461-4d65-437e-a801-09688f36b36f'}

In [138]:
res_time50_reads100.record

rec.array([([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1], -422., 2, 0.    ),
           ([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1], -415., 8, 0.0625),
           ([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1], -421., 1, 0.    ),
           ([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1], -421., 1, 0.    ),
           ([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1], -421., 1, 0.    ),
           ([0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1], -421., 1, 0.    ),
           ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1], -421., 2, 0.    ),
           ([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1], -421., 1, 0.    ),
           ([1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1], -421., 1, 0.    ),
           ([1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1], -421., 5, 0.    ),
           ([1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1], -416., 4, 0.0625),
           ([1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1], -420., 1, 0.    ),
           ([1, 1, 0, 1, 1, 

In [142]:
valid_y_list, valid_y_num_t1 = t1_9_find_valid_y(res_time50_reads100)

[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]
y is ok now
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1]
[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]
y is ok now
LI is ok too
drop out
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
y is ok now
[1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1]
y is ok now
LI is ok too
drop out
[0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1]
y is ok now
[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
y is ok now
LI is ok too
perfect
[1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]
y is ok now
[1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
[1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
[1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
y is ok now
[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1]
[1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]
y is ok now
[1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]
[1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 

In [143]:
valid_y_num_t1

1

In [144]:
valid_y_list

[[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]]

In [145]:
print(np.dot(pd.Series(valid_y_list[0]), ost16['LI']))
print(np.dot(pd.Series(valid_y_list[0]), ost16['SEX']))
print(np.dot(pd.Series(valid_y_list[0]), ost16['AOP']))

9
9
8


### annealing_time=10

In [175]:
res_time10_reads100 = qa_sampler.sample_qubo(qb[0], annealing_time=10, num_reads=100)
res_time10_reads100.info

{'timing': {'qpu_sampling_time': 22894,
  'qpu_anneal_time_per_sample': 10,
  'qpu_readout_time_per_sample': 198,
  'qpu_access_time': 33759,
  'qpu_access_overhead_time': 1477,
  'qpu_programming_time': 10865,
  'qpu_delay_time_per_sample': 21,
  'total_post_processing_time': 142,
  'post_processing_overhead_time': 142,
  'total_real_time': 33759,
  'run_time_chip': 22894,
  'anneal_time_per_run': 10,
  'readout_time_per_run': 198},
 'problem_id': '9f3b3d67-b72d-476d-bfd8-460e36ffeadb'}

In [176]:
res_time10_reads100.record

rec.array([([1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1], -419., 3, 0.125 ),
           ([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1], -422., 1, 0.    ),
           ([1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1], -412., 1, 0.0625),
           ([0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], -421., 1, 0.    ),
           ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0], -421., 7, 0.0625),
           ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0], -421., 4, 0.0625),
           ([0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], -420., 1, 0.    ),
           ([1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1], -420., 1, 0.    ),
           ([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1], -420., 5, 0.    ),
           ([1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1], -420., 1, 0.    ),
           ([1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], -420., 1, 0.    ),
           ([1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1], -420., 1, 0.    ),
           ([1, 1, 1, 1, 1, 

In [177]:
valid_y_list, valid_y_num_t1 = t1_9_find_valid_y(res_time10_reads100)

[1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1]
y is ok now
[1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
y is ok now
LI is ok too
drop out
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0]
y is ok now
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0]
y is ok now
[0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
y is ok now
[1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1]
[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1]
y is ok now
LI is ok too
drop out
[1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
y is ok now
[1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1]
[1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0]
y is ok now
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0]
y is ok now
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0]
y is ok now
[1,

In [178]:
valid_y_list

[[1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [179]:
valid_y_num_t1

1

## num_reads=1000

### annealing_time=20

In [150]:
res_reads1000 = qa_sampler.sample_qubo(qb[0], num_reads=1000)

In [151]:
valid_y_list, valid_y_num_t1 = t1_9_find_valid_y(res_reads1000)

[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
y is ok now
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1]
y is ok now
[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1]
y is ok now
[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1]
y is ok now
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1]
y is ok now
LI is ok too
perfect
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]
y is ok now
[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
y is ok now
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1]
y is ok now
[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
y is ok now
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1]
y is ok now
[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1]
y is ok now
LI is ok too
drop out
[1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1]
y is ok now
[1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]
y is ok now
LI is ok too
drop out
[1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1

In [152]:
valid_y_num_t1

2

### annealing_time=10

In [170]:
res_time10_reads1000 = qa_sampler.sample_qubo(qb[0], annealing_time=10, num_reads=1000)

In [171]:
valid_y_list, valid_y_num_t1 = t1_9_find_valid_y(res_time50_reads100)

[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]
y is ok now
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1]
[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]
y is ok now
LI is ok too
drop out
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
y is ok now
[1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1]
y is ok now
LI is ok too
drop out
[0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1]
y is ok now
[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
y is ok now
LI is ok too
perfect
[1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]
y is ok now
[1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
[1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
[1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
y is ok now
[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1]
[1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]
y is ok now
[1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]
[1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 

In [172]:
valid_y_list

[[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]]

In [173]:
valid_y_num_t1

1

In [174]:
### annealing time が 10よりも20のほうがよかった

### annealing_time = 20, num_reads=100, for t1 in range(0, ost16.shape[0]+1)

In [209]:
dw_sampler = DWaveSampler(
    endpoint="https://cloud.dwavesys.com/sapi",
    solver = 'DW_2000Q_6',
    token = DWAVE_TOKEN
)

In [210]:
qa_sampler = AutoEmbeddingComposite(dw_sampler)

In [211]:
def find_valid_y(num_reads):
    valid_y_list = {}
    valid_y_num = {}
    for t1 in range(0, ost16.shape[0]+1):
        y = Array.create('y', shape=ost16.shape[0], vartype='BINARY')

        #QUBO式で定式化
        H = (sum(y) - t_list[0])**2
        H += (np.dot(ost16[columns[1]], y) - t1)**2
        for j in range(2, len(columns)):
            H_plus = (np.dot(ost16[columns[j]], y) - t_list[j])**2
            H += H_plus

        qb = H.compile().to_qubo()
        res = qa_sampler.sample_qubo(qb[0], num_reads=100)

        valid_y_list[t1] = []
        valid_y_num[t1] = 0
        for y_info in list(res.record):
            if sum(y_info[0]) == t_list[0]:#int同士の比較
                y = pd.Series(y_info[0])
                if np.dot(ost16[columns[1]], y) == t1:#int同士の比較
                    for j in range(2, len(columns)):
                        if np.dot(ost16[columns[j]], y) != t_list[j]:#int同士の比較
                            break
                    else:
                        if all(list(y_info[0]) != p for p in valid_y_list[t1]):
                            valid_y_num[t1] += 1
                            valid_y_list[t1].append(list(y_info[0]))
                            print('perfect')
    return valid_y_list, valid_y_num

In [212]:
import time

# 処理前の時刻
t1 = time.time() 

In [213]:
%%timeit

valid_y_list, valid_y_num = find_valid_y(100)

perfect
perfect
perfect
perfect
perfect
perfect
perfect
perfect
perfect
perfect
perfect
perfect
perfect
perfect
perfect
perfect
perfect
perfect
perfect
perfect
perfect
perfect
perfect
perfect
perfect
perfect
perfect
17.2 s ± 522 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [214]:
# 処理後の時刻
t2 = time.time()

# 経過時間を表示
elapsed_time = t2-t1
print(f"経過時間：{elapsed_time}秒")

経過時間：142.7447226047516秒


In [215]:
valid_y_list

{0: [],
 1: [],
 2: [],
 3: [],
 4: [],
 5: [],
 6: [],
 7: [],
 8: [],
 9: [[1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1]],
 10: [[1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0]],
 11: [],
 12: [],
 13: [],
 14: [],
 15: [],
 16: []}

In [216]:
valid_y_num

{0: 0,
 1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 1,
 10: 3,
 11: 0,
 12: 0,
 13: 0,
 14: 0,
 15: 0,
 16: 0}

In [217]:
valid_y_num[9]/sum(valid_y_num.values())

0.25